Here we implement the algorithm to generate tree-unitary gates.

In [ ]:
import numpy as np

In [ ]:
q = 2
M = np.random.rand(q**3,q**3)
M_ = M.reshape([q]*6)

for _ in range(5000):
    #Unitarity
    M_ = M_.reshape(q**3,q**3)
    U,_,Vh = np.linalg.svd(M_, full_matrices=True)
    M_ = U @ Vh
    M_ = M_.reshape([q]*6)
    
    #First tree-unitarity
    M_ = np.einsum('abcdef->bcefad',M_)
    M_ = M_.reshape(q**4,q**2)
    U,S,Vh = np.linalg.svd(M_, full_matrices=True)
    M_ = U[:,:q**2] @ Vh
    M_ = M_.reshape([q]*6)*np.sqrt(2)
    M_ = np.einsum('bcefad->abcdef',M_)

    #Second tree-unitarity
    M_ = np.einsum('abcdef->acdfbe',M_)
    M_ = M_.reshape(q**4,q**2)
    U,S,Vh = np.linalg.svd(M_, full_matrices=True)
    M_ = U[:,:q**2] @ Vh
    M_ = M_.reshape([q]*6)*np.sqrt(2)
    M_ = np.einsum('acdfbe->abcdef',M_)
    
    #Third tree-unitarity
    M_ = np.einsum('abcdef->abdecf',M_)
    M_ = M_.reshape(q**4,q**2)
    U,S,Vh = np.linalg.svd(M_, full_matrices=True)
    M_ = U[:,:q**2] @ Vh
    M_ = M_.reshape([q]*6)*np.sqrt(2)
    M_ = np.einsum('abdecf->abcdef',M_)

In [ ]:
#Unitarity
U_0 = np.array(M_.reshape(q**3,q**3))
print(np.linalg.norm(np.conj(U_0.T) @ U_0   - np.eye(q**3)))

#Tree-unitarity
Id = np.eye(q)
P = np.einsum('ab,cd->abcd',Id,Id).reshape(q**2,q**2)

#Tree-unitarity 1
R_1 = np.einsum('aghcij,bghdij->abcd',M_,np.conj(M_)).reshape(q**2,q**2)
print(np.linalg.norm(R_1-2.*P))

#Tree-unitarity 2
R_2 = np.einsum('gahicj,gbhidj->abcd',M_,np.conj(M_)).reshape(q**2,q**2)
print(np.linalg.norm(R_2-2.*P))

#Tree-unitarity 2
R_3 = np.einsum('ghaijc,ghbijd->abcd',M_,np.conj(M_)).reshape(q**2,q**2)
print(np.linalg.norm(R_3-2.*P))

Note: occassionally this algorithm doesn't converge properly - just run again with different starting point!

In [ ]:
#Check additional properties
#Should only be zero when abc and def are grouped together
import itertools
for s in itertools.permutations(['a','b','c','d','e','f']):
    label = ''.join(s)
    N = np.einsum('abcdef->%s'%label,M_).reshape([q**3,q**3])
    print(label,":",np.linalg.norm(N @ np.conj(N.T) - np.eye(q**3)))

We can also generate a TU with a one or more maximum velocity conditions. For example, taking maximum velocity in the 2->1 direction:

In [ ]:
q = 2
M = np.random.rand(q**3,q**3)
M_ = M.reshape([q]*6)

for _ in range(5000):
    #Unitarity
    M_ = M_.reshape(q**3,q**3)
    U,_,Vh = np.linalg.svd(M_, full_matrices=True)
    M_ = U @ Vh
    M_ = M_.reshape([q]*6)
    
    #First tree-unitarity
    M_ = np.einsum('abcdef->bcefad',M_)
    M_ = M_.reshape(q**4,q**2)
    U,S,Vh = np.linalg.svd(M_, full_matrices=True)
    M_ = U[:,:q**2] @ Vh
    M_ = M_.reshape([q]*6)*np.sqrt(2)
    M_ = np.einsum('bcefad->abcdef',M_)


    #Second tree-unitarity as a max velocity condition 2->1
    M_ = np.einsum('abcdef->bceadf',M_)
    M_ = M_.reshape(q**3,q**3)
    U,S,Vh = np.linalg.svd(M_, full_matrices=True)
    M_ = U @ Vh
    M_ = M_.reshape([q]*6)*np.sqrt(2)
    M_ = np.einsum('bceadf->abcdef',M_)
    
    
    #Third tree-unitarity
    M_ = np.einsum('abcdef->abdecf',M_)
    M_ = M_.reshape(q**4,q**2)
    U,S,Vh = np.linalg.svd(M_, full_matrices=True)
    M_ = U[:,:q**2] @ Vh
    M_ = M_.reshape([q]*6)*np.sqrt(2)
    M_ = np.einsum('abdecf->abcdef',M_)

In [ ]:
#Unitarity
U_0 = np.array(M_.reshape(q**3,q**3))
print(np.linalg.norm(np.conj(U_0.T) @ U_0   - np.eye(q**3)))

#Tree-unitarity
Id = np.eye(q)
# Pt=np.einsum('ab,cd->abcd',Id,Id)
P = np.einsum('ab,cd->abcd',Id,Id).reshape(q**2,q**2)
# Q = np.einsum('abcd,ef->abcdef',Pt,Id).reshape(q**3,q**3)

#Tree-unitarity 1
R_1 = np.einsum('aghcij,bghdij->abcd',M_,np.conj(M_)).reshape(q**2,q**2)
print(np.linalg.norm(R_1-2.*P))

#Tree-unitarity 2
R_2 = np.einsum('gahicj,gbhidj->abcd',M_,np.conj(M_)).reshape(q**2,q**2)
print(np.linalg.norm(R_2-2.*P))

#Tree-unitarity 3
R_3 = np.einsum('ghaijc,ghbijd->abcd',M_,np.conj(M_)).reshape(q**2,q**2)
print(np.linalg.norm(R_3-2.*P))

#Max-velocity 2->3
R_4 = np.einsum('abijck,deijfk->abcdef',M_,np.conj(M_)).reshape(q**3,q**3)
print(np.linalg.norm(R_4-np.eye(q**3)))

#Max-velocity 2->1
R_4 = np.einsum('iabjck,idejfk->abcdef',M_,np.conj(M_)).reshape(q**3,q**3)
print(np.linalg.norm(R_4-np.eye(q**3)))